In [ ]:
from music21 import note as m21_note, chord as m21_chord, stream
from sklearn.neighbors import KNeighborsClassifier
import pygame
import time
import os
from training_data import training_data

# makes the melody into a vector the ML can understand
def notes_to_interval_vector(melody):
    midi = [m21_note.Note(n).pitch.midi for n in melody]
    return [midi[i+1] - midi[i] for i in range(len(midi)-1)]

# trains the k-nn model
X = [notes_to_interval_vector(mel) for mel, _ in training_data]
y = [label for _, label in training_data]
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X, y)

# for the harmony of the chords
def chord_to_notes(chord_label):
    chord_map = {
        "C": ["C3", "E3", "G3"],      "Cm": ["C3", "Eb3", "G3"],
        "C#": ["C#3", "F3", "G#3"],   "C#m": ["C#3", "E3", "G#3"],
        "D": ["D3", "F#3", "A3"],     "Dm": ["D3", "F3", "A3"],
        "D#": ["D#3", "G3", "A#3"],   "D#m": ["D#3", "F#3", "A#3"],
        "E": ["E3", "G#3", "B3"],     "Em": ["E3", "G3", "B3"],
        "F": ["F3", "A3", "C4"],      "Fm": ["F3", "Ab3", "C4"],
        "F#": ["F#3", "A#3", "C#4"],  "F#m": ["F#3", "A3", "C#4"],
        "G": ["G2", "B2", "D3"],      "Gm": ["G2", "Bb2", "D3"],
        "G#": ["G#2", "C3", "D#3"],   "G#m": ["G#2", "B2", "D#3"],
        "A": ["A2", "C#3", "E3"],     "Am": ["A2", "C3", "E3"],
        "A#": ["A#2", "D3", "F3"],    "A#m": ["A#2", "C#3", "F3"],
        "B": ["B2", "D#3", "F#3"],    "Bm": ["B2", "D3", "F#3"],
    }
    return chord_map.get(chord_label, ["C3", "E3", "G3"])

# prompts user for input
while True:
    user_input = input("🎹 Enter a melody (e.g. C4 E4 G4 F4), or 'q' to quit: ").strip()
    if user_input.lower() == "q":
        print("👋 Catch you next time!")
        break

    notes = user_input.split()
    if len(notes) < 3:
        print("❗️ Please enter at least 3 notes.")
        continue

    chunks = [notes[i:i+3] for i in range(0, len(notes), 3)]
    predicted_chords = []

    for chunk in chunks:
        if len(chunk) == 3:
            ivector = notes_to_interval_vector(chunk)
            prediction = model.predict([ivector])[0]
            probs = model.predict_proba([ivector])[0]
            print(f"\n🎶 For {chunk}: Predicted → {prediction}")
            for chord, prob in zip(model.classes_, probs):
                if prob > 0.01:
                    print(f"   ↳ {chord}: {round(prob * 100, 2)}%")
            predicted_chords.append(prediction)

    # makes the sounds together
    melody_stream = stream.Part()
    for pitch in notes:
        melody_stream.append(m21_note.Note(pitch, quarterLength=0.5))

    chord_stream = stream.Part()
    for label in predicted_chords:
        c = m21_chord.Chord(chord_to_notes(label), quarterLength=1.5)
        chord_stream.append(c)

    combined = stream.Stream()
    combined.append(chord_stream)
    combined.append(melody_stream)

    path = os.path.abspath("interval_live.mid")
    combined.write('midi', fp=path)

    # play the sounds
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(0.5)

C:\Users\sagpa\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


🎹 Enter a melody (e.g. C4 E4 G4 F4), or 'q' to quit:  f4 g4 a4 bb4 c5



🎶 For ['f4', 'g4', 'a4']: Predicted → C
   ↳ C: 66.67%
   ↳ Cm: 33.33%


🎹 Enter a melody (e.g. C4 E4 G4 F4), or 'q' to quit:  ab4 d#4 g4 c4



🎶 For ['ab4', 'd#4', 'g4']: Predicted → Cm
   ↳ Cm: 66.67%
   ↳ Fm: 33.33%


🎹 Enter a melody (e.g. C4 E4 G4 F4), or 'q' to quit:  f4 g#4 c5 g#4 c#5 g#4 b#4 g4



🎶 For ['f4', 'g#4', 'c5']: Predicted → Cm
   ↳ Cm: 66.67%
   ↳ Fm: 33.33%

🎶 For ['g#4', 'c#5', 'g#4']: Predicted → Cm
   ↳ Cm: 33.33%
   ↳ Em: 33.33%
   ↳ Fm: 33.33%
